# 2. Additive Manufacturing Path Planning Made Effortless

--> ***(before running this code, please consult Quick Start to make sure everything is set up)***

--> You will additionally need `pycalphad` and `pathfinding` libraries to run part of this exercise. If you are running this in Codespaces, it has been pre-installed for you.

**In this tutorial, we will demonstrate how effortless it is to dramatically speed up the exploration of feasible compositional spaces in high dimensional spaces through employing `nimplex`'s graph representations that abstract the underlying problem and dimensionality.**

**We will also design several neat, mathematically optimal (given some criteria) paths in a 7-component chemical space connecting two alloys of interest by mixing 4 fixed-composition alloy powders to create a tetrahedral attainable/design space. The beauty of this approach is that at no point (except for plotting in 3D for "human consumption") will we explicitly consider the dimensionality or the distance as the connectivity between the points in the space has been abstracted into graph adjacency. If you wish to add another alloy to the design process, you add it to the list, and you are done :)**

In [1]:
# Import nimplex and some of its plotting utilities
import nimplex
from utils import plotting

In [2]:
# Python wrapper for Plotly library
import plotly.express as px
import pandas as pd
from pprint import pprint
from copy import deepcopy

## Recall Last Example
Let's get back to our example (QuickStart) of the pair of **attainable space** simplex grid and corresponding **elemental space** positions defined for `7`-component elemental space of `Ti`, `Zr`, `Hf`, `W`, `Nb`, `Ta`, and `Mo` formed by `4` alloys:
- Ti50 Zr50 
- Hf95 Ti5
- Mo33 Nb33 Ta33
- Mo10 Nb10 W80

which we can represent as points in the elemental space:

In [3]:
elementalSpaceComponents = ['Ni', 'Cr', 'Fe']
attainableSpaceComponents = ['SS304L', 'Ni', 'Cr']
attainableSpaceComponentPositions = [[0.096114519430,0.1993865031, 0.7044989775], [100, 0, 0], [0, 100, 0]]

In [4]:
#ndims on 1 side + 1
# e.g. slen=12 creates a triangle with 13 points on each side
slen=24
dimsize = int(((slen+1)**2+(slen+1))/2)

And create tetrahedral grids with their compositions quantized at `12` divisions per dimension.

In [5]:
gridAtt, gridEl = nimplex.embeddedpair_simplex_grid_fractional_py(attainableSpaceComponentPositions, slen)

We used the **elemental**, or chemical, space to run the Root Mean Square Atomic Displacement (RMSAD) model by Tandoc (10.1038/s41524-023-00993-x) which acts as a lower-cost proxy for yield stress and hardness estimations in the absence of direct data:

import pqam_rmsadtandoc2023
rmsadList = []
for point in gridEl:
    formula = ' '.join([f'{c}{p}' for c, p in zip(elementalSpaceComponents, point)])
    rmsadList.append(pqam_rmsadtandoc2023.predict(formula))

And the **attainable** space grid to plot the results after projecting them to the Euclidean space:

# Hover approximate formula for each point
formulas = []
for i, comp in enumerate(gridEl):
    formulas.append(f"({i:>3}) "+"".join([f"{el}{100*v:.1f} " if v>0 else "" for el, v in zip(elementalSpaceComponents, comp)]))

# Generate the projected grid
gridAtt_projected_df = pd.DataFrame(plotting.simplex2cartesian_py(gridAtt), columns=['x','y'])

# Attach pure component (alloy) labels to corners
pureComponentIndices = nimplex.pure_component_indexes_py(3, 12)
labels = ['']*len(gridAtt_projected_df)
for comp, idx in zip(attainableSpaceComponents, pureComponentIndices):
    labels[idx] = "<b>"+comp+"</b>"

# Add text labels at the corners of the simplex
px.scatter(gridAtt_projected_df, x='x', y='y', color=rmsadList, text=labels, hover_name=formulas,
              template='plotly_white', width=800, height=700, 
              labels={'color':'RMSAD', 'x':'', 'y':'', 'z':''})

In [6]:
# Hover approximate formula for each point
formulas = []
for i, comp in enumerate(gridEl):
    formulas.append(f"({i:>3}) "+"".join([f"{el}{100*v:.1f} " if v>0 else "" for el, v in zip(elementalSpaceComponents, comp)]))

# Generate the projected grid
gridAtt_projected_df = pd.DataFrame(plotting.simplex2cartesian_py(gridAtt), columns=['x','y'])

# Attach pure component (alloy) labels to corners
pureComponentIndices = nimplex.pure_component_indexes_py(3, slen)
labels = ['']*len(gridAtt_projected_df)
for comp, idx in zip(attainableSpaceComponents, pureComponentIndices):
    labels[idx] = "<b>"+comp+"</b>"


## Thermodynamic Equilibria

**In this section we will be using `pycalphad` which is an amazing free open-source library for building thermodynamic models and performing calculations on them, including studying of phase equilibria, or what atomic arrangements are stable (coexist in equilibrium) at a given temperature and composition.** You can read more about `pycalphad` at [pycalphad.org](https://pycalphad.org/) and find an great tutorial at [this 2023 workshop materials repository](https://github.com/materialsgenomefoundation/2023-workshop-material/tree/main/pycalphad).

Let's start by loading a database of thermodynamic properties which defines, among other things, the Gibbs energy of each phase as a function of temperature and composition. You can find many of such databases at [TDBDB](https://avdwgroup.engin.brown.edu) maintained by Axel van de Walle's group at Brown University. 

In this example, we will be using a TDB `CrHfMoNbTaTiVWZr_9element_Feb2023` placed for you in the `examples` directory, which is a 9-element database for the elements `Cr`, `Hf`, `Mo`, `Nb`, `Ta`, `Ti`, `V`, `W`, and `Zr`, being developed by Shuang Lin in our group (Phases Research Lab at PSU). This wersion is an older *work in progress* one that has not been published, so while it is perfect for tutorial like this one, please refrain from using it for any serious work.

In [7]:
from pycalphad import Database
dbf = Database("ammap/databases/Cr-Fe-Ni_miettinen1999.tdb")
phases = list(set(dbf.phases.keys()))
print(elementalSpaceComponents)
print(f'Loaded TDB file with phases considered: {phases}')

['Ni', 'Cr', 'Fe']
Loaded TDB file with phases considered: ['SIGMA', 'HCP_A3', 'FCC_A1', 'BCC_A2', 'LIQUID']


As you can see, we will be looking at several different phases here. To keep things as simple as possible, we can split them in three groups:
- **liquid** phases: `LIQUID` (which we obviously want to avoid in a solid part design)
- **solid solution** phases: `FCC_A1`, `BCC_A2`, and `HCP_A3`
- **intermetallic** phases: `LAVES_C14`, `LAVES_C15`, and `LAVES_C36`

Designing an alloy is a complex procedure but for the sake of this tutorial, we will be focusing on the most common issue which is the formation of intermetallic phases which cause embrittlement and reduce the ductility of the material. **Thus, we will apply a simple constraint that the alloy should only contain the solid solution phases FCC, BCC, and HCP.**

**Now, knowing what we are looking for (phases at equilibrium), we can start by writing a short Python script around `pycalphad` to calculate the equilibrium phases for a given chemical elements composition `elP`. It is already placed in the `examples` directory as `myPycalphadCallable.py` which defines a function `equilibrium_callable` that takes a composition and returns the equilibrium phases.**

**We will arbitrarily pick `1000K`** as the temperature for the sake of this tutorial, but you can change it to any other value you like. Or even make it a list and add phases present at each temperature to a set to apply our constraint over a range of temperatures.


Please note that much more information is generated in the process (e.g., chemical composition of each phase and its fraction) but we are only interested in the phase presence. If you wish to do so, modifying the script to, e.g., allow for up to 5% of intermetallic phases, is a trivial task. Advanced users may also want to have a look at the `scheil_callable` we do not use in this tutorial for the sake of runtime, but which can be used to simulate solidification of the alloy from a liquid state in an additive manufacturing process.

In [8]:
from ammap.callables.EqScheil import equilibrium_callable

Let's test it on some composition in our space starting with the first point!

In [9]:
print(formulas[0])
equilibrium_callable(gridEl[0])

(  0) Cr100.0 


{'Phases': ['BCC_A2'], 'PhaseFraction': [0.9999999999974667]}

You should see `['BCC_A2']` in a second or so if you've run it at the default `1000K`. Quick and neat, right? Now, let's pick some compositionally complex alloy that does not lay around the corner of the attainable space tetrahedron and presents an actual challenge.

In [10]:
print(formulas[22])
equilibrium_callable(gridEl[22])

( 22) Ni91.7 Cr8.3 


{'Phases': ['FCC_A1'], 'PhaseFraction': [0.9999999999994575]}

Now, you should have seen an example of infeasible point composed of `['HCP_A3', 'LAVES_C15', 'BCC_A2']`. Let's deploy this in parallel over all the points in the elemental space `gridEl` and see how it looks like! We will use the `process_map` function from the `tqdm` library to show a neat progress bar while the calculations are running in parallel. On the 4-core Codespaces VM you can expect it to take around 2-3 minutes.

In [11]:
from tqdm import tqdm
from tqdm.contrib.concurrent import process_map

In [12]:
gridPhases = process_map(equilibrium_callable, gridEl)

  0%|          | 0/325 [00:00<?, ?it/s]

Let's see how some of the data looks like.

In [13]:
len(gridPhases)

325

Now, let's turn that list of phases into a list of feasibility based on the constraint we defined earlier. Note that in some cases, the `pycalphad` library may return an empty list of phases, which we will treat as infeasible.

infeasiblePhases = set([
    'AL3M', 'AL5FE2', 'LIQUID', 'AL11CR4', 'D82_217', 'AL3NI1', 'NI3TI', 'AL5FE4', 'TAOA', 'LAVES_C15', 'GAMMAH', 'AL5CR', 'AL2FE', 'AL4CR', 'NIV3_A15', 'GAMMA_H', 
    'ALTI3_D019', 'ALCU_DELTA', 'AL45CR7', 'TS01TI', 'NI8V', 'AL45CR7_12', 'LAVES_C14', 'LAVES_A15', 'AL3NI5', 'AL5TI3', 'ALCU_EPSILON', 'NITI2', 'AL8V5', 'ALCR2', 'AL23V4', 'AL13FE4', 
    'AL4NI3', 'ALCU_ZETA', 'ALCU_THETA', 'AL3NI2', 'AL23V4_194', 'CRNI2', 'NI3V_D022', 'LAVES_C36', 'TS01T3', 'AL21V2_227', 'AL5TI2', 'ALCU_ETA', 'GAMMAL', 
    'GAMMA_D83', 'SIGMA_D8B', 'AL21V2', 'AL2TI', 'AL3TI_L', 'ALTI', 'AL45V7'])
print(f"Feasible phases set: {set(phases).difference(infeasiblePhases)}")


In [14]:
infeasiblePhases = set(['LIQUID', 'SIGMA'])
print(f"Feasible phases set: {set(phases).difference(infeasiblePhases)}")

Feasible phases set: {'BCC_A2', 'HCP_A3', 'FCC_A1'}


In [15]:
feasiblePhases = set(['FCC_A1', 'BCC_A2', 'HCP_A3', 'BCC_B2', 'FCC_L12', 'L12_FCC'])


In [16]:
for i in range(len(gridPhases)):
    print(len(set(gridPhases[i]['Phases']).difference(feasiblePhases)))


0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
1
1
0
0
0
0
0
0
0
0
0
0
0
1
1
1
1
1
0
0
0
0
0
0
0
0
0
1
1
1
1
1
0
0
0
0
0
0
0
0
1
1
1
1
1
1
0
0
0
0
0
0
1
1
1
1
1
1
1
0
0
0
0
1
1
1
1
1
1
1
0
0
0
1
1
1
1
1
1
0
0
0
1
1
1
1
1
1
0
0
1
1
1
1
1
0
0
1
1
1
1
0
0
1
1
1
1
0
1
1
1
0
1
1
0
1
0
1


In [30]:
gridFeasibleTest = [len(gridPhases[i]['Phases']) != 0 and len(set(gridPhases[i]['Phases']).difference(feasiblePhases)) == 0 and gridPhases[i] != [] for i in range(len(gridPhases))]


for phase in gridPhases:
    phase['infeasiblePhases'] = sum(phase['PhaseFraction'][i] for i, p in enumerate(phase['Phases']) if p in infeasiblePhases)
phases1 = [gp['Phases'] for gp in gridPhases]
sum1= [gridPhases[i]['infeasiblePhases'] for i in range(len(gridPhases))]
gridFeasible = [len(set(p) & infeasiblePhases)==0 and p!=[] for p in phases1]

gridFeasible = [len(set(p) & infeasiblePhases)==0 and p!=[] for p in phases1]


In [31]:
len(gridFeasibleTest)

325

Finally, let's plot the result in 3D using the `plotly` library and our spacial-transformed attainable space grid we obtained with `plotting.simplex2cartesian_py(gridAtt)` earlier.

**Once you run the cell below, you should be seeing an interactive 3D plot with 455 split roughly 50/50 between feasible and infeasible points. You can rotate the plot, zoom in and out, and hover over the points to see their composition and feasibility. You can also click on the legend to hide/show the points based on their feasibility.**

In [32]:
color_map = {'path':'red', True:'#FECB52', False:'blue'}

In [34]:
fig = px.scatter(gridAtt_projected_df, x='x', y='y', color=gridFeasibleTest, text=labels, hover_name=formulas,
              template='plotly_white', width=800, height=600, color_discrete_map=color_map, 
              labels={'color':'Solid Solution Phases', 'x':'', 'y':''})
fig.update_traces( marker_size=12)
fig.show()

In [35]:
from pathfinding.core.graph import Graph
from pathfinding.finder.dijkstra import DijkstraFinder

In [36]:
finder = DijkstraFinder()

# Feasibility Gliding

In [118]:
formulas = []
for i, comp in enumerate(gridEl):
    formulas.append(f"({i:>3}) "+"".join([f"{el}{100*v:.1f} " if v>0 else "" for el, v in zip(elementalSpaceComponents, comp)]))

In [119]:
_, _, graphN = nimplex.embeddedpair_simplex_graph_fractional_py(attainableSpaceComponentPositions, slen)

In [120]:
graphN[:5]

[[1, 25], [0, 2, 25, 26], [1, 3, 26, 27], [2, 4, 27, 28], [3, 5, 28, 29]]

In [121]:
formulas[dimsize-1]

'(324) Ni9.6 Cr19.9 Fe70.4 '

In [122]:
startingNode = 0
endNode = dimsize-1 

print(f"Starting node: {formulas[startingNode]}")
print(f"Ending node: {formulas[endNode]}")

Starting node: (  0) Cr100.0 
Ending node: (324) Ni9.6 Cr19.9 Fe70.4 


In [123]:
gridFeasible = [None]*len(graphN)
queue = [startingNode]
explored = set()
calcCount = 0

In [124]:
while len(queue)>0:
    print(f"Queue: {queue}")
    # Assign feasibilities to the current queue
    elPositions = [gridEl[i] for i in queue]
    if len(queue)>3:
        phases = process_map(equilibrium_callable, elPositions, max_workers=4)
    else:
        phases = [equilibrium_callable(elP) for elP in elPositions]
    feasibilities = [len(gridPhases[i]['Phases']) != 0 and len(set(phases[i]['Phases']).difference(feasiblePhases)) == 0 and phases[i] != [] for i in range(len(phases))]
    #feasibilities = [len(set(p['Phases']) & infeasiblePhases)==0 and p['Phases']!=[] for p in phases]
    feasibilities[0]=True

    calcCount += len(feasibilities)
    explored = explored.union(queue)

    # Create next queue based on neighbors of feasible points
    nextQueue = set()
    for f, i in zip(feasibilities, queue):
        gridFeasible[i] = f
        # Only if feasible
        if f:
            for n in graphN[i]:
                if n not in explored:
                    nextQueue.add(n)

    # Early termination criteria if we just evaluated the target
    if endNode in queue:
        break

    print(f"Calculations done: {calcCount}")
    queue = list(nextQueue)

Queue: [0]
Calculations done: 1
Queue: [1, 25]
Calculations done: 3
Queue: [49, 2, 26]
Calculations done: 6
Queue: [72, 27, 50, 3]


  0%|          | 0/4 [00:00<?, ?it/s]

Calculations done: 10
Queue: [4, 73, 51, 28, 94]


  0%|          | 0/5 [00:00<?, ?it/s]

Calculations done: 15
Queue: [5, 74, 115, 52, 29, 95]


...

Calculations done: 242
Queue: [258, 134, 269, 279, 24, 153, 285, 288, 294, 295, 296, 171, 48, 188, 71, 204, 219, 93, 233, 114, 246]


  0%|          | 0/21 [00:00<?, ?it/s]

Calculations done: 263
Queue: [302, 303]
Calculations done: 265
Queue: [309, 308, 301]
Calculations done: 268
Queue: [313, 314, 307]
Calculations done: 271
Queue: [312, 317, 318]
Calculations done: 274
Queue: [321, 306, 316, 311]


  0%|          | 0/4 [00:00<?, ?it/s]

Calculations done: 278
Queue: [320, 323]
Calculations done: 280
Queue: [322, 324, 319]


In [125]:
fig = px.scatter(gridAtt_projected_df, x='x', y='y', color=gridFeasible, text=labels, hover_name=formulas,
              template='plotly_white', width=800, height=600, color_discrete_sequence=['green', 'red'],
              labels={'color':'Solid Solution Phases', 'x':'', 'y':''})
fig.show()

## Multi-TDB

- elementalSpaceComponents = ["Ti", "Zr", "Hf", "W", "Nb", "Ta", "Mo"]
### System 1
- attainableSpaceComponents = ["Ti50 Zr50", "Mo33 Nb33 Ta33", "Mo80 Nb10 W10"]
- attainableSpaceComponentPositions = [[50, 50, 0, 0, 0, 0, 0], [0, 0, 0, 33, 33, 33, 0], [0, 0, 0, 10, 10, 0, 80]]
### System 2
- attainableSpaceComponents = ["Ti50 Zr50", "Mo33 Nb33 Ta33", "Hf95 Zr5"]
- attainableSpaceComponentPositions = [[50, 50, 0, 0, 0, 0, 0], [0, 0, 0, 33, 33, 33, 0], [0, 5, 95, 0, 0, 0, 0]]

In [82]:
def system_graphs(systemIdx, attainableSpaceComponentPositions, ndiv=slen):
    gridAtt, gridEl, graphN = nimplex.embeddedpair_simplex_graph_fractional_py(attainableSpaceComponentPositions, ndiv)
    gridPhases = process_map(equilibrium_callable, gridEl)
    #gridFeasible = [len(set(p['Phases']) & infeasiblePhases)==0 and p!=[] for p in gridPhases]
    gridFeasible = [len(gridPhases[i]['Phases']) != 0 and len(set(gridPhases[i]['Phases']).difference(feasiblePhases)) == 0 and gridPhases[i] != [] for i in range(len(gridPhases))]
    gridFeasible[0]=True # Note the hardcoded idx 0 feasibility!
    edges = []
    offset = len(graphN)*systemIdx
    for i, nList in enumerate(graphN):
        if gridFeasible[i] or i==0: 
            for n in nList:
                if gridFeasible[n]:
                    edges.append([i+offset, n+offset, 1])
    return gridAtt, gridEl, graphN, edges, gridFeasible, gridPhases

In [83]:
gridAtt1, gridEl1, graphN1, edges1, gridFeasible1, gridPhases1 = system_graphs(0, [[100, 0, 0], [0, 100, 0],[0.096114519430,0.1993865031, 0.7044989775]])

  0%|          | 0/325 [00:00<?, ?it/s]

In [84]:
pureComponentIndices = nimplex.pure_component_indexes_py(3, slen)
labels = ['']*len(gridAtt_projected_df)
for comp, idx in zip(["Ni", "Cr", "SS304L"], pureComponentIndices):
    labels[idx] = "<b>"+comp+"</b>"
fig = px.scatter(gridAtt_projected_df, x='x', y='y', color=gridFeasible1, text=labels, 
              template='plotly_white', width=800, height=600, color_discrete_map=color_map, 
              labels={'color':'Solid Solution Phases', 'x':'', 'y':''})
fig.show()

In [85]:
from ammap.callables.EqScheil2 import equilibrium_callable2
from ammap.callables.EqScheil1 import equilibrium_callable1
from ammap.callables.EqScheil1 import elementalSpaceComponents as comp1

In [86]:
def system_graphs1(systemIdx, attainableSpaceComponentPositions, ndiv=slen):
    gridAtt, gridEl, graphN = nimplex.embeddedpair_simplex_graph_fractional_py(attainableSpaceComponentPositions, ndiv)
    gridPhases = process_map(equilibrium_callable1, gridEl)
    #gridFeasible = [len(set(p['Phases']) & infeasiblePhases)==0 and p!=[] for p in gridPhases]
    gridFeasible = [len(gridPhases[i]['Phases']) != 0 and len(set(gridPhases[i]['Phases']).difference(feasiblePhases)) == 0 and gridPhases[i] != [] for i in range(len(gridPhases))]
    edges = []
    offset = len(graphN)*systemIdx
    for i, nList in enumerate(graphN):
        if gridFeasible[i]:
            for n in nList:
                if gridFeasible[n]:
                    edges.append([i+offset, n+offset, 1])
    return gridAtt, gridEl, graphN, edges, gridFeasible, gridPhases
def system_graphs2(systemIdx, attainableSpaceComponentPositions, ndiv=slen):
    gridAtt, gridEl, graphN = nimplex.embeddedpair_simplex_graph_fractional_py(attainableSpaceComponentPositions, ndiv)
    gridPhases = process_map(equilibrium_callable2, gridEl)
    #gridFeasible = [len(set(p['Phases']) & infeasiblePhases)==0 and p!=[] for p in gridPhases]
    gridFeasible = [len(gridPhases[i]['Phases']) != 0 and len(set(gridPhases[i]['Phases']).difference(feasiblePhases)) == 0 and gridPhases[i] != [] for i in range(len(gridPhases))]
    edges = []
    offset = len(graphN)*systemIdx
    for i, nList in enumerate(graphN):
        if gridFeasible[i]:
            for n in nList:
                if gridFeasible[n]:
                    edges.append([i+offset, n+offset, 1])
    return gridAtt, gridEl, graphN, edges, gridFeasible, gridPhases

In [87]:
gridAtt2, gridEl2, graphN2, edges2, gridFeasible2, gridPhases2 = system_graphs1(1, [[100,0,0], [0,100,0], [0,0,100]])

  0%|          | 0/325 [00:00<?, ?it/s]

In [88]:
pureComponentIndices = nimplex.pure_component_indexes_py(3, slen)
labels = ['']*len(gridAtt_projected_df)
for comp, idx in zip(["Ni", "Cr", "V"], pureComponentIndices):
    labels[idx] = "<b>"+comp+"</b>"
fig = px.scatter(gridAtt_projected_df, x='x', y='y', color=gridFeasible2, text=labels, 
              template='plotly_white', width=800, height=600, color_discrete_map=color_map, 
              labels={'color':'Solid Solution Phases', 'x':'', 'y':''})
fig.show()

In [89]:
gridAtt3, gridEl3, graphN3, edges3, gridFeasible3, gridPhases3 = system_graphs2(2, [[100,0,0], [0,100,0], [0,0,100]])
from ammap.callables.EqScheil2 import elementalSpaceComponents as comp2

  0%|          | 0/325 [00:00<?, ?it/s]

In [90]:
from pycalphad import Database
check=Database('ammap/databases/Cr-Ti-V_ghosh2002.tdb')
checkphase=check.phases.keys()
print(checkphase)


dict_keys(['LIQUID', 'BCC_A2', 'HCP_A3', 'LAVES_C14', 'LAVES_C36', 'LAVES_C15'])


In [40]:
#infeasiblePhases2 = set(['LIQUID', 'LAVES_C14', 'LAVES_C36', 'LAVES_C15'])

In [41]:
#gridFeasible3 = [len(set(p['Phases']) & infeasiblePhases2)==0 and p!=[] for p in gridPhases3]


In [91]:
pureComponentIndices = nimplex.pure_component_indexes_py(3, slen)
labels = ['']*len(gridAtt_projected_df)
for comp, idx in zip(comp2, pureComponentIndices):
    labels[idx] = "<b>"+comp+"</b>"
fig = px.scatter(gridAtt_projected_df, x='x', y='y', color=gridFeasible3, text=labels, 
              template='plotly_white', width=800, height=600, color_discrete_map=color_map,
              hover_name=list(range(len(gridAtt_projected_df))), 
              labels={'color':'Solid Solution Phases', 'x':'', 'y':''})
fig.show()

In [92]:
def stitch_edge(excludedComponent1Idx, excludedComponent2Idx, systemIdx1, systemIdx2, ndiv=slen):
    edge1, edge2 = [], []
    stitchEdges = []
    gridAtt = nimplex.simplex_grid_py(3, ndiv)
    offset1 = len(gridAtt)*systemIdx1
    offset2 = len(gridAtt)*systemIdx2
    for i, p in enumerate(gridAtt):
        if p[excludedComponent1Idx]==0:
            edge1.append(i+offset1)
        if p[excludedComponent2Idx]==0:
            edge2.append(i+offset2)
    assert len(edge1) == len(edge2)
    assert len(edge1)>0
    print(edge1)
    print(edge2)
    for i, j in zip(edge1, edge2):
        stitchEdges.append([i,j,1])
        stitchEdges.append([j,i,1])
    print(stitchEdges)
    return stitchEdges

In [93]:
stitch12 = stitch_edge(2,2,1,0)
stitch23 = stitch_edge(1,0,2,1)

[349, 373, 396, 418, 439, 459, 478, 496, 513, 529, 544, 558, 571, 583, 594, 604, 613, 621, 628, 634, 639, 643, 646, 648, 649]
[24, 48, 71, 93, 114, 134, 153, 171, 188, 204, 219, 233, 246, 258, 269, 279, 288, 296, 303, 309, 314, 318, 321, 323, 324]
[[349, 24, 1], [24, 349, 1], [373, 48, 1], [48, 373, 1], [396, 71, 1], [71, 396, 1], [418, 93, 1], [93, 418, 1], [439, 114, 1], [114, 439, 1], [459, 134, 1], [134, 459, 1], [478, 153, 1], [153, 478, 1], [496, 171, 1], [171, 496, 1], [513, 188, 1], [188, 513, 1], [529, 204, 1], [204, 529, 1], [544, 219, 1], [219, 544, 1], [558, 233, 1], [233, 558, 1], [571, 246, 1], [246, 571, 1], [583, 258, 1], [258, 583, 1], [594, 269, 1], [269, 594, 1], [604, 279, 1], [279, 604, 1], [613, 288, 1], [288, 613, 1], [621, 296, 1], [296, 621, 1], [628, 303, 1], [303, 628, 1], [634, 309, 1], [309, 634, 1], [639, 314, 1], [314, 639, 1], [643, 318, 1], [318, 643, 1], [646, 321, 1], [321, 646, 1], [648, 323, 1], [323, 648, 1], [649, 324, 1], [324, 649, 1]]
[650, 675

In [94]:
edgesCombined = edges1 + edges2 + stitch12
edgesCombined2 = edges2 + edges3 + stitch23
trueEdges = edges1 + edges2 + edges3 + stitch12 + stitch23

In [95]:
print(len(trueEdges))
print(len(edgesCombined))
print(len(edges1))
print(len(edges2))
print(len(edges3))

2802
1888
1354
484
864


In [96]:
pathfindingGraph = Graph(edges=deepcopy(trueEdges), bi_directional=False)

In [97]:
finder = DijkstraFinder()

In [98]:
len(pathfindingGraph.nodes)

540

In [99]:
path, runs = finder.find_path(
    pathfindingGraph.node(0), 
    pathfindingGraph.node(674),
    pathfindingGraph)

In [100]:
shortestPath = [p.node_id for p in path]

In [101]:
formulas = []
for i, comp in enumerate(gridEl1):
    formulas.append(f"({i:>3}) "+"".join([f"{el}{100*v:.1f} " if v>0 else "" for el, v in zip(elementalSpaceComponents, comp)]))
for i, comp in enumerate(gridEl2):
    formulas.append(f"({i+dimsize:>3}) "+"".join([f"{el}{100*v:.1f} " if v>0 else "" for el, v in zip(comp1, comp)]))
for i, comp in enumerate(gridEl3):
    formulas.append(f"({i+dimsize:>3}) "+"".join([f"{el}{100*v:.1f} " if v>0 else "" for el, v in zip(comp2, comp)]))

In [102]:
for step, i in enumerate(shortestPath):
    print(f"{step+1:>2}: {formulas[i]}")

 1: (  0) Ni9.6 Cr19.9 Fe70.4 
 2: ( 25) Ni13.4 Cr19.1 Fe67.5 
 3: ( 49) Ni17.1 Cr18.3 Fe64.6 
 4: ( 72) Ni20.9 Cr17.4 Fe61.6 
 5: ( 94) Ni24.7 Cr16.6 Fe58.7 
 6: ( 95) Ni24.3 Cr20.0 Fe55.8 
 7: (116) Ni28.0 Cr19.1 Fe52.8 
 8: (136) Ni31.8 Cr18.3 Fe49.9 
 9: (137) Ni31.4 Cr21.6 Fe47.0 
10: (156) Ni35.2 Cr20.8 Fe44.0 
11: (157) Ni34.8 Cr24.1 Fe41.1 
12: (158) Ni34.4 Cr27.5 Fe38.2 
13: (140) Ni30.2 Cr31.6 Fe38.2 
14: (141) Ni29.8 Cr35.0 Fe35.2 
15: (122) Ni25.6 Cr39.1 Fe35.2 
16: (123) Ni25.2 Cr42.5 Fe32.3 
17: (124) Ni24.8 Cr45.8 Fe29.4 
18: (104) Ni20.7 Cr50.0 Fe29.4 
19: (105) Ni20.3 Cr53.3 Fe26.4 
20: (106) Ni19.9 Cr56.6 Fe23.5 
21: (107) Ni19.5 Cr60.0 Fe20.5 
22: (108) Ni19.1 Cr63.3 Fe17.6 
23: (109) Ni18.7 Cr66.7 Fe14.7 
24: (110) Ni18.3 Cr70.0 Fe11.7 
25: (111) Ni17.9 Cr73.3 Fe8.8 
26: (112) Ni17.5 Cr76.7 Fe5.9 
27: (113) Ni17.1 Cr80.0 Fe2.9 
28: (114) Ni16.7 Cr83.3 
29: (439) Ni16.7 Cr83.3 
30: (417) Ni12.5 Cr83.3 V4.2 
31: (394) Ni8.3 Cr83.3 V8.3 
32: (370) Ni4.2 Cr83.3 V12.5 
3

In [103]:
gridFeasibleMarked3 = ['path' if i in shortestPath else f for i, f in enumerate(gridFeasible1)]

In [104]:
gridFeasibleMarked4 = ['path' if i in shortestPath else f for i, f in enumerate(gridFeasible2, start=dimsize)]

In [105]:
gridFeasibleMarked5 = ['path' if i in shortestPath else f for i, f in enumerate(gridFeasible3, start=2*dimsize)]

In [106]:
pureComponentIndices = nimplex.pure_component_indexes_py(3, slen)
labels = ['']*len(gridAtt_projected_df)
for comp, idx in zip(["Ni", "Cr", "SS"], pureComponentIndices):
    labels[idx] = "<b>"+comp+"</b>"
fig = px.scatter(gridAtt_projected_df, x='x', y='y', color=gridFeasibleMarked3, text=labels, 
                 hover_name=list(range(len(gridAtt_projected_df))),
              template='plotly_white', width=800, height=600, color_discrete_map=color_map)#, 
              #labels={'color':'Solid Solution Phases', 'x':'', 'y':''})
fig.show()

In [107]:
pureComponentIndices = nimplex.pure_component_indexes_py(3, slen)
labels = ['']*len(gridAtt_projected_df)
for comp, idx in zip(comp1, pureComponentIndices):
    labels[idx] = "<b>"+comp+"</b>"
fig = px.scatter(gridAtt_projected_df, x='x', y='y', color=gridFeasibleMarked4, text=labels, 
                 hover_name=[f"{i+dimsize}" for i in range(len(gridAtt_projected_df))],
              template='plotly_white', width=800, height=600, color_discrete_map=color_map) 
              #labels={'color':'Solid Solution Phases', 'x':'', 'y':''})
fig.show()

In [108]:
pureComponentIndices = nimplex.pure_component_indexes_py(3, slen)
labels = ['']*len(gridAtt_projected_df)
for comp, idx in zip(comp2, pureComponentIndices):
    labels[idx] = "<b>"+comp+"</b>"
fig = px.scatter(gridAtt_projected_df, x='x', y='y', color=gridFeasibleMarked5, text=labels, 
                 hover_name=[f"{i+dimsize*2}" for i in range(len(gridAtt_projected_df))],
              template='plotly_white', width=800, height=600, color_discrete_map=color_map) 
              #labels={'color':'Solid Solution Phases', 'x':'', 'y':''})
fig.show()

In [109]:
# Initialize the list to store the enumerated path as tuples
enumeratedPath = []

# Iterate through shortestPath and save the step and formula as tuples
for step, i in enumerate(shortestPath):
    enumeratedPath.append((step + 1, formulas[i]))

# Now enumeratedPath contains tuples with the step number and the corresponding formula
print(enumeratedPath)

[(1, '(  0) Ni9.6 Cr19.9 Fe70.4 '), (2, '( 25) Ni13.4 Cr19.1 Fe67.5 '), (3, '( 49) Ni17.1 Cr18.3 Fe64.6 '), (4, '( 72) Ni20.9 Cr17.4 Fe61.6 '), (5, '( 94) Ni24.7 Cr16.6 Fe58.7 '), (6, '( 95) Ni24.3 Cr20.0 Fe55.8 '), (7, '(116) Ni28.0 Cr19.1 Fe52.8 '), (8, '(136) Ni31.8 Cr18.3 Fe49.9 '), (9, '(137) Ni31.4 Cr21.6 Fe47.0 '), (10, '(156) Ni35.2 Cr20.8 Fe44.0 '), (11, '(157) Ni34.8 Cr24.1 Fe41.1 '), (12, '(158) Ni34.4 Cr27.5 Fe38.2 '), (13, '(140) Ni30.2 Cr31.6 Fe38.2 '), (14, '(141) Ni29.8 Cr35.0 Fe35.2 '), (15, '(122) Ni25.6 Cr39.1 Fe35.2 '), (16, '(123) Ni25.2 Cr42.5 Fe32.3 '), (17, '(124) Ni24.8 Cr45.8 Fe29.4 '), (18, '(104) Ni20.7 Cr50.0 Fe29.4 '), (19, '(105) Ni20.3 Cr53.3 Fe26.4 '), (20, '(106) Ni19.9 Cr56.6 Fe23.5 '), (21, '(107) Ni19.5 Cr60.0 Fe20.5 '), (22, '(108) Ni19.1 Cr63.3 Fe17.6 '), (23, '(109) Ni18.7 Cr66.7 Fe14.7 '), (24, '(110) Ni18.3 Cr70.0 Fe11.7 '), (25, '(111) Ni17.9 Cr73.3 Fe8.8 '), (26, '(112) Ni17.5 Cr76.7 Fe5.9 '), (27, '(113) Ni17.1 Cr80.0 Fe2.9 '), (28, '(114) N

import csv

# Assuming enumeratedPath is already defined
# Example: enumeratedPath = [(1, 'O2'), (2, 'CH4'), (3, 'C3H8')]

# Define the CSV file name
csv_file = 'enumerated_path.csv'

# Open the CSV file in write mode
with open(csv_file, mode='w', newline='') as file:
    writer = csv.writer(file)
    
    # Write the header (optional)
    writer.writerow(['Step', 'Formula'])
    
    # Write the data
    writer.writerows(enumeratedPath)

print(f"Data has been exported to {csv_file}")